Basic Statistics:

    Total Kills (K) per player. (By scope, by side, ) 
    Total Deaths (D) per player.
    Kill/Death ratio (K/D) per player.
    Average Combat Score (ACS) per player.
    Average Damage per Round (ADR) per player.

Performance Metrics:

    Headshot Percentage (HS%) per player.
    First Kills (FK) and First Deaths (FD) counts and ratios.
    Kill/Death difference (+/-) per player.
    Kill/Death difference in pistol rounds (+/-) per player.

Agent-Specific Metrics:

    Agent popularity and performance analysis.
    Win rates and performance on specific maps for each agent.

Team Performance:

    Total team Kills, Deaths, and K/D ratio.
    Team Average Combat Score (ACS) and Average Damage per Round (ADR).
    Team First Kills (FK) and First Deaths (FD) counts and ratios.

Map Performance:

    Individual player performance on each map.
    Team performance on each map.
    Win rates on each map.

Stage and Series Analysis:

    Performance in different stages (e.g., Playoffs, Grand Final).
    Series-level statistics and win rates.

Comparative Analysis:

    Compare individual player statistics against the team average.
    Compare agent performance across players.

Trend Analysis:

    Analyze how player performance changes over different maps or series.
    Identify trends in individual player K/D ratios.

Impact Analysis:

    Assess the impact of individual player performance on team success.


In [1]:
import pandas as pd
from utils import *

In [5]:
data = pd.read_csv('Data/stat_match_ascension.csv')
report = number_action_by_player_by_scope(df = data, action='K', average=True, team_name="Gentle Mates", side='Total')

In [24]:
import pandas as pd

df = pd.read_csv('champions-tour-2024-pacific-kickoff_data/general_data_champions-tour-2024-pacific-kickoff.csv')



In [31]:
# Match rate
data = df.drop_duplicates(['Stage', 'Series','Team Name'])
played = data[data['Team Name'] == "Paper Rex"].dropna()
wins = data[data['Team Name'] == "Paper Rex"].where(data['Team Name'] == data['winner']).dropna()
win_rate_matches = round(len(wins) / len(played), 2)

In [48]:
data_round = df.drop_duplicates(['Stage', 'Series','Team Name', 'Map #'])[['Stage', 'Series', 'Map #','Team Name', 'rounds', 'winner']]
data_round.head(2)

,Stage,Series,Map #,Team Name,rounds,winner
0,Group Stage,Opening (A),2,T1,"1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0...",T1
5,Group Stage,Opening (A),2,BLEED,"0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1...",T1


In [73]:
data_round.groupby(['Stage', 'Series','Team Name','Map #'], group_keys=True)[['winner','rounds']]
data_round['rounds'] = data_round['rounds'].apply(lambda x: list(map(int, x.split(', '))))
data_round['total_rounds'] = data_round['rounds'].apply(lambda x: sum(x))

In [77]:
data_round.head(10)

,Stage,Series,Map #,Team Name,rounds,winner,total_rounds
0,Group Stage,Opening (A),2,T1,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...",T1,13
5,Group Stage,Opening (A),2,BLEED,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...",T1,6
10,Group Stage,Opening (C),1,Gen.G,"[0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, ...",Gen.G,13
15,Group Stage,Opening (C),1,Rex Regum Qeon,"[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ...",Gen.G,11
20,Group Stage,Opening (C),2,Gen.G,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, ...",Gen.G,11
25,Group Stage,Opening (C),2,Rex Regum Qeon,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...",Gen.G,13
30,Group Stage,Opening (C),3,Gen.G,"[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",Gen.G,13
35,Group Stage,Opening (C),3,Rex Regum Qeon,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",Gen.G,8
40,Group Stage,Opening (A),1,ZETA DIVISION,"[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, ...",ZETA DIVISION,13
45,Group Stage,Opening (A),1,Global Esports,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, ...",ZETA DIVISION,10


In [83]:
def get_map_winner(match):
    # Group by map number and find the maximum rounds won
    map_rounds = match.groupby('Map #')['total_rounds'].max()
    
    # Get the map numbers where maximum rounds are achieved
    max_rounds_maps = map_rounds[map_rounds == map_rounds.max()].index
    
    # Check if there is only one map with maximum rounds
    if len(max_rounds_maps) == 1:
        # Get the match winner of that map
        map_winner = match[match['total_rounds'] == map_rounds.max()]['winner'].iloc[0]
    else:
        # If multiple maps have the same maximum rounds, return None
        map_winner = None
    
    return map_winner

# Apply the function to each match separately
data['Map Winner'] = data.groupby('Id').apply(get_map_winner)

C:\Users\marius.reymauzaize\AppData\Local\Temp\ipykernel_39660\135674916.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Map Winner'] = data.groupby('Id').apply(get_map_winner)


In [86]:
data.groupby('Id').head()

,Player Name,Agent Name,R,ACS,K,D,K/D +/–,+/–,ADR,HS%,...,Team Name,Map Name,Map #,Stage,Series,winner,Id,rounds,total_rounds,Map Winner
0,Carpe,Fade,1.47\n1.23\n1.89,294\n247\n374,20\n10\n10,\n\n10\n6\n4\n\n,8\n6\n2,+10\n+4\n+6,187\n159\n237,20%\n20%\n20%,...,T1,Lotus,2,Group Stage,Opening (A),T1,0,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...",13,T1
5,crazyguy,Breach,1.20\n1.29\n1.15,260\n307\n233,16\n7\n9,\n\n16\n6\n10\n\n,8\n4\n4,0\n+1\n-1,153\n198\n127,38%\n53%\n28%,...,BLEED,Lotus,2,Group Stage,Opening (A),T1,0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...",6,Paper Rex
10,t3xture,Jett,1.54\n1.32\n1.75,377\n339\n416,33\n15\n18,\n\n16\n8\n8\n\n,1\n1\n0,+17\n+7\n+10,245\n233\n256,31%\n37%\n22%,...,Gen.G,Icebox,1,Group Stage,Opening (C),Gen.G,1,"[0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, ...",13,Gen.G
15,xffero,Viper,1.21\n1.39\n1.03,269\n342\n198,22\n14\n8,\n\n17\n8\n9\n\n,12\n6\n6,+5\n+6\n-1,176\n233\n119,28%\n31%\n21%,...,Rex Regum Qeon,Icebox,1,Group Stage,Opening (C),Gen.G,1,"[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ...",11,NaN
40,SugarZ3ro,Brimstone,1.23\n1.90\n0.61,228\n339\n127,19\n14\n5,\n\n14\n5\n9\n\n,10\n6\n4,+5\n+9\n-4,153\n208\n102,26%\n30%\n20%,...,ZETA DIVISION,Bind,1,Group Stage,Opening (A),ZETA DIVISION,2,"[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, ...",13,NaN
45,Benkai,Cypher,0.92\n0.73\n1.12,156\n138\n177,13\n7\n6,\n\n15\n8\n7\n\n,4\n1\n3,-2\n-1\n-1,112\n101\n124,30%\n24%\n38%,...,Global Esports,Bind,1,Group Stage,Opening (A),ZETA DIVISION,2,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, ...",10,NaN
60,Jremy,Jett,1.09\n0.89\n1.34,230\n186\n289,16\n8\n8,\n\n15\n10\n5\n\n,6\n3\n3,+1\n-2\n+3,157\n134\n188,29%\n30%\n27%,...,Team Secret,Breeze,1,Group Stage,Opening (B),Talon Esports,3,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, ...",8,NaN
65,Governor,Neon,1.35\n0.35\n2.10,282\n102\n418,21\n2\n19,\n\n13\n9\n4\n\n,10\n5\n5,+8\n-7\n+15,176\n75\n252,17%\n21%\n16%,...,Talon Esports,Breeze,1,Group Stage,Opening (B),Talon Esports,3,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, ...",13,NaN
80,BuZz,Raze,1.23\n1.18\n1.27,269\n213\n313,20\n6\n14,\n\n16\n6\n10\n\n,4\n4\n0,+4\n0\n+4,174\n163\n183,24%\n29%\n21%,...,DRX,Lotus,1,Group Stage,Opening (B),DRX,4,"[1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, ...",13,NaN
85,Meiy,Raze,1.04\n1.37\n0.61,252\n301\n189,16\n12\n4,\n\n18\n10\n8\n\n,7\n4\n3,-2\n+2\n-4,173\n202\n134,20%\n26%\n9%,...,DetonatioN FocusMe,Lotus,1,Group Stage,Opening (B),DRX,4,"[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, ...",8,NaN
